In [15]:
%matplotlib inline
import matplotlib.pyplot as plt

import pickle 
import numpy as np
import torch
import os
from timeit import default_timer as timer
from torch.autograd import Variable

from sklearn import manifold
from sklearn.manifold.t_sne import trustworthiness
from sklearn.model_selection import KFold
from sklearn import neighbors
from sklearn.metrics import accuracy_score

In [2]:
from core.SimpleConvNet import SimpleConvNet
from core.GraphConvNet2 import GraphConvNet2
from core.DataEmbeddingGraph import DataEmbeddingGraph
from util.plot_embedding import plot_embedding, plot_embedding_subplot
from util.mnist_data_loader import get_train_set, get_test_set


In [13]:
if torch.cuda.is_available():
    print('cuda available')
    device = 'gpu'
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
else:
    print('cuda not available')
    device = 'cpu'
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor


cuda not available


In [4]:
task_parameters = {}
task_parameters['reduction_method'] = 'spectral'
task_parameters['n_components'] = 2

net_parameters = {}
net_parameters['n_components'] = task_parameters['n_components']
net_parameters['D'] = 784 # input dimension
net_parameters['H'] = 50 # number of hidden units
net_parameters['L'] = 10 # number of hidden layers
net_parameters['n_channels'] = 1
net_parameters['n_units_1'] = net_parameters['n_units_2'] = 64

In [5]:
filename = 'data/set_100_mnist_spectral_size_200_500.pkl'
with open(filename, 'rb') as f:
    [all_test_data] = pickle.load(f)

In [32]:
# Loss functions
def l2_norm(v):
    return np.sqrt(np.sum(np.square(v), axis=1, dtype=np.float64))

def pairwise_loss_function_1(y_true, y_pred, W):
    pairwise_loss_1 = mean_squared_error(y_true[W.row,:], y_true[W.col,:])
    pairwise_loss_2 = mean_squared_error(y_pred[W.row,:], y_pred[W.col,:])
    return np.square(pairwise_loss_1 - pairwise_loss_2)

def pairwise_loss_function_2(y_true, y_pred, W):
    pairwise_loss_1 = l2_norm(y_true[W.row,:] - y_true[W.col,:])
    pairwise_loss_2 = l2_norm(y_pred[W.row,:] - y_pred[W.col,:])
    return np.average(np.square(pairwise_loss_1 - pairwise_loss_2))

In [110]:
def evaluate_net(net, all_test_data):
    n_test = len(all_test_data)
    similarity_tracker = np.zeros((n_test, ))
    trust_tracker = np.zeros((n_test, ))
    one_nn_tracker = np.zeros((n_test, ))
    time_tracker = np.zeros((n_test, ))
    
    for i in range(n_test):
        G = all_test_data[i]
        time_start = timer()
        y_pred = net.forward(G).detach().numpy()
        time_end = timer()
        time_tracker[i] = time_end - time_start
        
        similarity_tracker[i] =  pairwise_loss_function_2(G.target, y_pred, G.adj_matrix)
        X = G.data.view(G.data.shape[0], -1).numpy()
        trust_tracker[i] = trustworthiness(X, y_pred, n_neighbors=5)
        one_nn_tracker[i] = one_nearest_neighbours_generalisation_error(y_pred, G.labels.numpy())
    return similarity_tracker, trust_tracker, one_nn_tracker, time_tracker

In [44]:
# Graph net
net = GraphConvNet2(net_parameters)
root = 'results/mnist_spectral1/'
filename = root + 'graph_net5.pkl'
checkpoint = torch.load(filename, map_location=device)
net.load_state_dict(checkpoint['state_dict'])

In [53]:
G = all_test_data[2]
y_pred = net.forward(G)

In [54]:
loss = pairwise_loss_function_2(y_pred.detach().numpy(), G.target, G.adj_matrix)
print(loss)

0.012579493418748878


In [55]:
y_pred.requires_grad

True

In [57]:
y_true = Variable(torch.FloatTensor(G.target).type(dtypeFloat), requires_grad=False)

In [60]:
def torch_pairwise_loss(y, y_target, W):
    distances_1 = y_target[W.row,:] - y_target[W.col,:]
    distances_2 = y[W.row,:] - y[W.col,:]
    loss = torch.mean(torch.pow(distances_1.norm(dim=1) - distances_2.norm(dim=1), 2))
    return loss

In [61]:
loss = torch_pairwise_loss(y_pred, y_true, G.adj_matrix)
print(loss)

tensor(0.0126, grad_fn=<MeanBackward1>)
